In [1]:
from __future__ import print_function, division, unicode_literals
import six
import os, sys
from os.path import join
import json
from codecs import open
from collections import defaultdict
from operator import itemgetter
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
import codecs
import random
from time import time

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

from nltk.corpus import stopwords

In [2]:
import lucene
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.document import Document, Field
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexReader
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.search import Sort, SortField
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.store import SimpleFSDirectory
from org.apache.lucene.util import Version
from java.io import File
lucene.initVM()

In [3]:
DATA_DIR = join(os.environ['HOME'], 'data/allen-ai-challenge')
WIKI_DIR = join(DATA_DIR, 'wiki_dump')
CK12_DIR = join(DATA_DIR, 'ck12_dump')
TRAINING_SET = join(DATA_DIR, 'training_set.tsv')
VALIDATION_SET = join(DATA_DIR, 'validation_set.tsv')
TRAINING_SET_MERGED = join(DATA_DIR, 'training_set_merged.tsv')
# INDEX_DIR = join(DATA_DIR, 'index-wiki-ck12')
# INDEX_DIR = join(DATA_DIR, 'index-ck12-stem')
# INDEX_DIR = join(DATA_DIR, 'index-all-l_stem_summ')
INDEX_DIR = join(DATA_DIR, 'index-ck12-stem')
SUBMISSION = join(DATA_DIR, 'submissions/lucene_wiki_ck12_17jan.tsv')
VOCABULARY = join(DATA_DIR, 'vocabulary', 'w2v_a2_5.tsv')
SENT_DELIM = ' | '

In [4]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english') + '. , ! ? !? ?! ... ; : - —'.split())
def cleanup_text(text):
    sents = []
    for s in nltk.sent_tokenize(text):        
#         words = [stemmer.stem(w) for w in nltk.word_tokenize(s.lower()) if w not in stopwords]
        words = [w for w in nltk.word_tokenize(s.lower()) if w not in stopwords]
        if words:
            sents.append(words)
    return SENT_DELIM.join([' '.join(s) for s in sents])
#     return [stemmer.stem(w.text) for w in nlp(text) if not w.is_stop and not w.is_punct and not w.is_space]

In [5]:
%%time
vocab = {}
with open(VOCABULARY, encoding='utf8') as f:
    for word, vec in (line.strip().split('\t', 1) for line in f):
        vocab[word] = np.fromstring(vec, sep='\t')
vocab_dim = 300

CPU times: user 1.47 s, sys: 20 ms, total: 1.49 s
Wall time: 1.7 s


In [6]:
vocab['earth'].shape

(300,)

In [7]:
cleanup_text('Here is some text. What the heck more place?')

u'text | heck place'

Index Creation
----------

In [38]:
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
writerConfig = IndexWriterConfig(Version.LUCENE_4_10_1, StandardAnalyzer())
writer = IndexWriter(SimpleFSDirectory(File(INDEX_DIR)), writerConfig)

In [39]:
def add_document(doc_text):
    doc = Document()
    doc.add(Field("text", cleanup_text(doc_text), Field.Store.YES, Field.Index.ANALYZED))
    writer.addDocument(doc)

In [10]:
%%time

# CK12
for i, fn_short in enumerate(os.listdir(CK12_DIR)):
    fn = join(CK12_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        ck12_article = json.load(f)
#         content = []
#         for subtitle, paragraph in ck12_article['contents'].items():
#             content.append(subtitle + '. ' + paragraph)
#         add_document(' '.join(content))    

        for subtitle, paragraph in ck12_article['contents'].items():
            add_document(subtitle + '. ' + paragraph)

CPU times: user 25.7 s, sys: 232 ms, total: 26 s
Wall time: 33.4 s


In [40]:
%%time

# Wiki
for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
    fn = join(WIKI_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        wiki_article = json.load(f)
        _, summary, content = wiki_article
        add_document(summary)
        for p in re.sub('\[ \d* \]', ' ', content).replace('\n\n\n', '.\n ').split('\n\n'):
            add_document(p)

CPU times: user 5min 17s, sys: 1.91 s, total: 5min 19s
Wall time: 7min 19s


In [41]:
doc_count = writer.numDocs()
writer.close()
doc_count

243469

Build NN
----------

In [72]:
# %%time
# VERSION 0
# import theano
# import theano.tensor as T
# import lasagne
# import lasagne.layers as LL
# from lasagne.nonlinearities import elu, rectify

# M = 1.0

# input_context = LL.InputLayer((None, 300))
# input_hyp = LL.InputLayer((None, 300))

# l_diff = LL.ElemwiseMergeLayer([input_context, input_hyp], merge_function=T.sub)
# l_mult = LL.ElemwiseMergeLayer([input_context, input_hyp], merge_function=T.mul)

# nn = LL.concat([l_diff, l_mult])
# nn = LL.DenseLayer(nn, 100, nonlinearity=elu)
# nn = LL.DenseLayer(nn, 1, nonlinearity=rectify)
# t_output = LL.get_output(nn)[:, 0]

# t_must_be_less = t_output[0::2]
# t_must_be_more = t_output[1::2]

# # t_cost = (T.maximum(0, M + t_must_be_less - t_must_be_more)).mean()
# t_cost = (T.maximum(0, M + T.sqr(t_must_be_less) - T.sqr(t_must_be_more))).mean()
# # t_cost = (T.sqr(t_must_be_less) + T.sqr(T.maximum(0, M - t_must_be_more))).mean()  # square-square
# # t_cost = (T.sqr(t_must_be_less) + T.exp(-t_must_be_more)).mean()  # square-exponential

# params = LL.get_all_params(nn)

# updates = lasagne.updates.adam(t_cost, params)

# train_fn = theano.function([input_hyp.input_var, input_context.input_var], t_cost, updates=updates)
# cost_fn = theano.function([input_hyp.input_var, input_context.input_var], t_cost)
# energy_fn = theano.function([input_hyp.input_var, input_context.input_var], t_output)

CPU times: user 1.77 s, sys: 16 ms, total: 1.79 s
Wall time: 1.78 s


In [107]:
%%time
# VERSION 1
import theano
import theano.tensor as T
import lasagne
import lasagne.layers as LL
from lasagne.nonlinearities import elu, rectify

M = 1.0

LEN_CONTEXT = 50
LEN_HYP = 20

input_context = LL.InputLayer((None, LEN_CONTEXT, 300))
input_context_mask = LL.InputLayer((None, LEN_CONTEXT))
l_c = LL.LSTMLayer(input_context, 300, only_return_final=True, mask_input=input_context_mask)

input_hyp = LL.InputLayer((None, LEN_HYP, 300))
input_hyp_mask = LL.InputLayer((None, LEN_HYP))
l_h = LL.LSTMLayer(input_hyp, 300, only_return_final=True, mask_input=input_hyp_mask)

nn = LL.concat([l_c, l_h])
nn = LL.DenseLayer(nn, 100, nonlinearity=elu)
nn = LL.DenseLayer(nn, 1, nonlinearity=rectify)
t_output = LL.get_output(nn)[:, 0]

t_must_be_less = t_output[0::2]
t_must_be_more = t_output[1::2]

# t_cost = (T.maximum(0, M + t_must_be_less - t_must_be_more)).mean()
t_cost = (T.maximum(0, M + T.sqr(t_must_be_less) - T.sqr(t_must_be_more))).mean()
# t_cost = (T.sqr(t_must_be_less) + T.sqr(T.maximum(0, M - t_must_be_more))).mean()  # square-square
# t_cost = (T.sqr(t_must_be_less) + T.exp(-t_must_be_more)).mean()  # square-exponential

params = LL.get_all_params(nn)

updates = lasagne.updates.adam(t_cost, params)

train_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_cost, updates=updates)
cost_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_cost)
energy_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_output)

CPU times: user 34.9 s, sys: 1.35 s, total: 36.3 s
Wall time: 1min 42s


In [ ]:
%%time
# VERSION 1
import theano
import theano.tensor as T
import lasagne
import lasagne.layers as LL
from lasagne.nonlinearities import elu, rectify

M = 1.0

LEN_CONTEXT = 50
LEN_HYP = 20

input_context = LL.InputLayer((None, LEN_CONTEXT, 300))
input_context_mask = LL.InputLayer((None, LEN_CONTEXT))
l_c = LL.LSTMLayer(input_context, 300, only_return_final=True, mask_input=input_context_mask)

input_hyp = LL.InputLayer((None, LEN_HYP, 300))
input_hyp_mask = LL.InputLayer((None, LEN_HYP))
l_h = LL.LSTMLayer(input_hyp, 300, only_return_final=True, mask_input=input_hyp_mask)

nn = LL.concat([l_c, l_h])
nn = LL.DenseLayer(nn, 100, nonlinearity=elu)
nn = LL.DenseLayer(nn, 1, nonlinearity=rectify)
t_output = LL.get_output(nn)[:, 0]

t_must_be_less = t_output[0::2]
t_must_be_more = t_output[1::2]

# t_cost = (T.maximum(0, M + t_must_be_less - t_must_be_more)).mean()
t_cost = (T.maximum(0, M + T.sqr(t_must_be_less) - T.sqr(t_must_be_more))).mean()
# t_cost = (T.sqr(t_must_be_less) + T.sqr(T.maximum(0, M - t_must_be_more))).mean()  # square-square
# t_cost = (T.sqr(t_must_be_less) + T.exp(-t_must_be_more)).mean()  # square-exponential

params = LL.get_all_params(nn)

updates = lasagne.updates.adam(t_cost, params)

train_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_cost, updates=updates)
cost_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_cost)
energy_fn = theano.function([input_hyp.input_var, input_hyp_mask.input_var,
                            input_context.input_var, input_context_mask.input_var], t_output)

Read index
-----------

In [108]:
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

In [109]:
indexi = np.arange(reader.maxDoc())

In [118]:
def take_random(seq, drop=True):
    '''Changes seq!'''
    idx = random.randint(0, len(seq)-1)
    d = seq[idx]
    if drop:
        del seq[idx]
    return d, seq    

def corrupt_context(index, window_nearest, window_farest, target='context', modify_context=True):    
    artice = reader.document(index)['text']
    sentsA = artice.split(SENT_DELIM)
    if len(sentsA) < 2:
        return None, None
    hypA, restA = drop_random(sentsA, drop=modify_context)
    others = [i for i in range(index - window_farest, index + window_farest + 1) 
              if (np.abs(index-i) > window_nearest) and 
                 (i >= 0) and (i < doc_count)]
    idxB = random.choice(others)
    sentsB = reader.document(idxB)['text'].split(SENT_DELIM)
    if target == 'context':
        return [(hypA, ' '.join(restA)), (hypA, ' '.join(sentsB))]
    elif target == 'hyp':
        hypB, restB = take_random(sentsB, drop=False)
        return [(hypA, ' '.join(restA)), (hypB, ' '.join(restA))]
    else:
        raise RuntimeError('Unknown corruption target = %s' % context)


def mean_w2v(text):
    vec = np.zeros((vocab_dim,), dtype='float64')
    c = 1
    for w in nltk.word_tokenize(text):
        if w in vocab:
            vec += vocab[w]
            c += 1
    return (vec/c).astype('float32'), None


def seq_w2v(text, max_len):
    row = 0
    words = list(nltk.word_tokenize(text))
    n = max(max_len, len(words))
    
    vec = np.zeros((n, vocab_dim), dtype='float32')
    mask = np.zeros((n,), dtype='int')

    for w in words:
        if w in vocab:
            vec[row] = vocab[w]
            mask[row] = 1
            row += 1
    return vec[-max_len:], mask[-max_len:]
    

In [111]:
%%time
questions = []
with open(TRAINING_SET, encoding='utf8') as f:
    f.readline()
    for line in f:
        qid, q, correct, aa, ab, ac, ad = line.strip().split('\t')

        query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", cleanup_text(q)))
        hits = searcher.search(query, 20).scoreDocs
        doc = reader.document(hits[0].doc)['text']
        q_vec, q_mask = seq_w2v(doc, LEN_CONTEXT)
        vecs_context = np.repeat(q_vec[np.newaxis], 4, axis=0)
        vecs_context_mask = np.repeat(q_mask[np.newaxis], 4, axis=0)
        vecs_hyp = np.zeros((4, LEN_HYP, vocab_dim), dtype='float32')
        vecs_hyp_mask = np.zeros((4, LEN_HYP), dtype='int')
        for i, a in enumerate([aa, ab, ac, ad]):
            try:
                query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", cleanup_text(a)))
                hits = searcher.search(query, 20).scoreDocs
                doc = reader.document(hits[0].doc)['text']
            except:
                doc = ''
            vecs_hyp[i], vecs_hyp_mask[i] = seq_w2v(doc, LEN_HYP)
            questions.append((qid, 'ABCD'.index(correct), vecs_hyp, vecs_hyp_mask, vecs_context, vecs_context_mask))


def check(questions):
    scores = []
    for qid, idx_correct, vecs_hyp, vecs_hyp_mask, vecs_context, vecs_context_mask in questions:
        energies = energy_fn(vecs_hyp, vecs_hyp_mask, vecs_context, vecs_context_mask)
        scores.append(np.argmin(energies) == idx_correct)
    return np.mean(scores)

CPU times: user 25.1 s, sys: 1.27 s, total: 26.4 s
Wall time: 52.5 s


In [122]:
def randomized_cyclic_generator(seq):
    rseq = list(seq)
    while True:
        random.shuffle(rseq)
        for s in rseq:
            yield s

In [123]:
# %%time

BATCH = 20
EPOCHS = 100
            
id_generator = iter(randomized_cyclic_generator(np.arange(doc_count, dtype=int)))

corruption_targets = ['context'] + ['hyp']
drop_hyp_from_context = False
win_min = 5
win_width = 5

print_each_sec = 30

time_started = time()
# acc = check(questions)
acc = 0
costs = []
rows_seen = 0
while True:
    batch_hyp = np.zeros((BATCH*2, LEN_HYP, vocab_dim), dtype='float32')
    batch_hyp_m = np.zeros((BATCH*2, LEN_HYP), dtype=int)
    batch_context = np.zeros((BATCH*2, LEN_CONTEXT, vocab_dim), dtype='float32')
    batch_context_m = np.zeros((BATCH*2, LEN_CONTEXT), dtype=int)

    b = 0
    while b < BATCH:
        idx_row_in_db = int(next(id_generator))
        right, corrupted = corrupt_context(idx_row_in_db, min_win, win_min + win_width, 
                                           random.choice(corruption_targets), modify_context=drop_hyp_from_context)
        if right is None:
            continue

        batch_hyp[b*2], batch_hyp_m[b*2] = seq_w2v(right[0], LEN_HYP)
        batch_context[b*2], batch_context_m[b*2] = seq_w2v(right[1], LEN_CONTEXT)

        batch_hyp[b*2+1], batch_hyp_m[b*2+1] = seq_w2v(corrupted[0], LEN_HYP)
        batch_context[b*2+1], batch_context_m[b*2+1] = seq_w2v(corrupted[1], LEN_CONTEXT)
        b += 1
            
    batch_cost = train_fn(batch_hyp, batch_hyp_m, batch_context, batch_context_m)
#     print(batch_cost)
    costs.append(batch_cost)
    rows_seen += BATCH
    if time() - time_started > print_each_sec:
        print('%d: %.3f (%.2f%%) in %.0fs' % (rows_seen, np.mean(costs), acc * 100, time() - time_started))
        sys.stdout.flush()
        time_started = time()
#         acc = check(questions)
        acc = 0
        costs = []    
    

TypeError: drop_random() got an unexpected keyword argument 'drop'